In [18]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math

 Se hará una red neuronal que reciba como input el coeficiente de absorción y como output tengamos radio, fracción volumen y espesor de la película

# Juntando todos los archivos en uno solo

In [19]:
files = [file for file in os.listdir('./parametros')]

param = pd.DataFrame()
for file in files:
  df = pd.read_csv('./parametros/'+file)
  param = pd.concat([param, df], axis= 0)

In [20]:
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,301,0.016685,1,10,100
1,302,0.016167,1,10,100
2,303,0.015703,1,10,100
3,304,0.015290,1,10,100
4,305,0.014916,1,10,100
...,...,...,...,...,...
19195,896,0.002255,1,9,120
19196,897,0.002257,1,9,120
19197,898,0.002258,1,9,120
19198,899,0.002260,1,9,120


# Revolviendo todos los datos

Esto es para el entrenamiento óptimo de la red y poderle dar una mayor gama de datos

In [21]:
param = param.sample(frac=1, random_state=42).reset_index(drop=True)


In [22]:
param

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
0,753,0.001533,1,23,120
1,310,0.014692,3,10,120
2,625,0.002815,1,10,85
3,604,0.000572,1,9,120
4,373,0.003104,1,21,120
...,...,...,...,...,...
38395,566,0.000711,1,10,70
38396,785,0.000418,12,10,120
38397,659,0.000772,1,9,120
38398,561,0.005686,1,10,105


In [23]:
from sklearn.model_selection import train_test_split


# Separando nuestra base de datos en 2, una que servirá para entrenar el modelo y otra para hacerle un test

In [24]:
train_df, test_df = train_test_split(param, test_size=0.2, random_state=42)

test será el 20% de los datos mientras train el 80%

In [25]:
test_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
27452,492,0.006226,5,10,120
15009,831,0.001993,1,26,120
8124,700,0.000512,1,26,120
38103,748,0.000418,11,10,120
25872,432,0.002877,1,10,65
...,...,...,...,...,...
3746,820,0.002003,7,10,120
18195,422,0.000481,1,7,120
37286,551,0.033743,4,10,120
15361,703,0.000239,1,29,120


In [26]:
train_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
36772,825,0.001564,11,10,120
22714,538,0.004552,1,10,65
3031,442,0.000189,1,10,120
9632,391,0.000021,1,10,115
29786,870,0.002197,1,11,120
...,...,...,...,...,...
6265,760,0.002772,1,10,100
11284,623,0.001487,1,32,120
38158,836,0.001522,1,10,135
860,324,0.008735,1,6,120


# Prueba 1

In [27]:
from tensorflow import keras
from tensorflow.keras import layers

In [28]:
data = train_df
input_data = data.drop(['Fracvol', 'Longitud de onda', 'Radio (nm)', 'Espesor de la película (nm)'],axis=1).values


In [29]:

fracvol = data.drop(['Índice de absorción', 'Longitud de onda', 'Radio (nm)', 'Espesor de la película (nm)'],axis=1).values
radio = data.drop(['Índice de absorción', 'Longitud de onda', 'Fracvol', 'Espesor de la película (nm)'],axis=1).values
esp_pel = data.drop(['Índice de absorción', 'Longitud de onda', 'Fracvol', 'Radio (nm)'],axis=1).values

In [30]:
inputs = keras.Input(shape=(3,), name='entrada')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
output1_layer = layers.Dense(1, name='fracvol')(x)
output2_layer = layers.Dense(1, name='radio')(x)
output3_layer = layers.Dense(1, name='esp_pel')(x)

In [31]:
model = keras.Model(inputs=inputs, outputs=[output1_layer, output2_layer, output3_layer])

# Compilar el modelo
model.compile(optimizer='adam',
              loss={'output1': 'binary_crossentropy', 'output2': 'binary_crossentropy', 'output3': 'binary_crossentropy'},
              metrics=['accuracy'])

In [32]:
# Entrenar el modelo
model.fit(input_data, {'fracvol': fracvol, 'radio': radio, 'esp_pel': esp_pel}, epochs=100, batch_size=1)


Epoch 1/100


ValueError: in user code:

    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\PcR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1023, in train_step
   

No sirvió tratando de hacer fit el modelo

# Prueba 2

In [33]:
# Crear un DataFrame de entrada y salida
data = param

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# Obtener los datos de entrada y salida para el conjunto de entrenamiento y prueba
train_input_data = train_data['Índice de absorción'].values
train_output_data = train_data[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']].values
test_input_data = test_data['Índice de absorción']
test_output_data = test_data[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]



In [34]:
# Crear el modelo
inputs = keras.Input(shape=(1,), name='input')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)


In [35]:
output1_layer = layers.Dense(1, name='Fracvol')(x)
output3_layer = layers.Dense(1, name='espesor_pelicula_nm')(x)
output2_layer = layers.Dense(1, name='Radio_nm')(x)


In [ ]:
model = keras.Model(inputs=inputs, outputs=[output1_layer, output2_layer, output3_layer])

# Compilar el modelo
model.compile(optimizer='adam',
              loss={'Fracvol': 'binary_crossentropy', 'Radio_nm': 'binary_crossentropy', 'espesor_pelicula_nm': 'binary_crossentropy'},metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_input_data, train_output_data, epochs=2, batch_size=1)

Epoch 1/2
30720/30720 [==============================] - 29s 909us/step - loss: -1952.5455 - Fracvol_loss: -650.4683 - Radio_nm_loss: -650.7025 - espesor_pelicula_nm_loss: -651.3922 - Fracvol_accuracy: 0.2532 - Radio_nm_accuracy: 0.2532 - espesor_pelicula_nm_accuracy: 0.2535
Epoch 2/2
30720/30720 [==============================] - 28s 900us/step - loss: -1974.8123 - Fracvol_loss: -658.2731 - Radio_nm_loss: -658.2731 - espesor_pelicula_nm_loss: -658.2731 - Fracvol_accuracy: 0.2554 - Radio_nm_accuracy: 0.2554 - espesor_pelicula_nm_accuracy: 0.2554


In [36]:
arr = train_data['Índice de absorción'].to_numpy()

In [ ]:
predictions = model.predict(arr)
predictions

960/960 [==============================] - 1s 689us/step


[array([[1.2751088],
        [1.2610755],
        [1.2561675],
        ...,
        [1.258758 ],
        [1.2622546],
        [1.2558494]], dtype=float32),
 array([[1.4972441],
        [1.4780445],
        [1.4713296],
        ...,
        [1.4748737],
        [1.4796579],
        [1.4708942]], dtype=float32),
 array([[1.7655803],
        [1.7428479],
        [1.7348975],
        ...,
        [1.7390937],
        [1.7447582],
        [1.7343822]], dtype=float32)]

In [ ]:
posit = []
for fila in predictions:
    for num in fila:
        if num > 0:
            posit.append(num)

Nos da números pero no tienen significado físico, como negativos

# Prueba 3

In [37]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

In [38]:
# Cargar DataFrame
dataframe = train_df
# Separar datos de entrada y salida
input_data = dataframe['Índice de absorción']
output_data = dataframe[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

# Crear modelo de red neuronal
model = Sequential()

# Agregar capa de entrada
model.add(Dense(units=8, input_dim=1, activation='relu'))

# Agregar capas ocultas
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=16, activation='relu'))

# Agregar capa de salida
model.add(Dense(units=3, activation='sigmoid'))

# Compilar modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar modelo
model.fit(input_data, output_data, epochs=10, batch_size=32)

Epoch 1/10
960/960 [==============================] - 1s 678us/step - loss: -88269.9688 - accuracy: 1.0000
Epoch 2/10
960/960 [==============================] - 1s 652us/step - loss: -2244709.7500 - accuracy: 1.0000
Epoch 3/10
960/960 [==============================] - 1s 651us/step - loss: -12089685.0000 - accuracy: 1.0000
Epoch 4/10
960/960 [==============================] - 1s 637us/step - loss: -35962456.0000 - accuracy: 1.0000
Epoch 5/10
960/960 [==============================] - 1s 642us/step - loss: -79876328.0000 - accuracy: 1.0000
Epoch 6/10
960/960 [==============================] - 1s 660us/step - loss: -149883616.0000 - accuracy: 1.0000
Epoch 7/10
960/960 [==============================] - 1s 651us/step - loss: -252241344.0000 - accuracy: 1.0000
Epoch 8/10
960/960 [==============================] - 1s 679us/step - loss: -393667392.0000 - accuracy: 1.0000
Epoch 9/10
960/960 [==============================] - 1s 670us/step - loss: -581321792.0000 - accuracy: 1.0000
Epoch 10/1

In [39]:
arr = train_data['Índice de absorción'].to_numpy()

Haciendo predicciones

In [40]:
predictions = model.predict(arr)
predictions

960/960 [==============================] - 1s 502us/step


array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

Quitando los unos de las predicciones

In [ ]:
indices = np.where(predictions != 1)
indices

(array([], dtype=int64), array([], dtype=int64))

Nos da puros unos como resultados

# Prueba 4

In [41]:
# Cargar DataFrame
df = train_df
# Separar datos de entrada y salida
X = df['Índice de absorción']
Y = df[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']]

In [42]:
# Convertimos los dataframes en numpy arrays
X = np.array(X).reshape(-1, 1)
y = np.array(Y)

In [93]:
# Creamos el modelo de la red neuronal
model = Sequential()
model.add(Dense(64, input_dim=1, activation='relu'))
model.add(Dense(3, activation='linear'))

In [134]:
model.compile(loss='mean_absolute_error', optimizer='adam')

In [135]:
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
768/768 [==============================] - 1s 808us/step - loss: 4.8802 - val_loss: 4.8216
Epoch 2/100
768/768 [==============================] - 1s 706us/step - loss: 4.7890 - val_loss: 4.8092
Epoch 3/100
768/768 [==============================] - 1s 680us/step - loss: 4.7845 - val_loss: 4.8046
Epoch 4/100
768/768 [==============================] - 1s 683us/step - loss: 4.7804 - val_loss: 4.8054
Epoch 5/100
768/768 [==============================] - 1s 686us/step - loss: 4.7769 - val_loss: 4.7980
Epoch 6/100
768/768 [==============================] - 1s 688us/step - loss: 4.7734 - val_loss: 4.7992
Epoch 7/100
768/768 [==============================] - 1s 690us/step - loss: 4.7702 - val_loss: 4.7922
Epoch 8/100
768/768 [==============================] - 1s 695us/step - loss: 4.7670 - val_loss: 4.7930
Epoch 9/100
768/768 [==============================] - 1s 695us/step - loss: 4.7642 - val_loss: 4.7882
Epoch 10/100
768/768 [==============================] - 1s 697us/step - l

# Predicciones del modelo

In [136]:

arr = test_df['Índice de absorción'].to_numpy()

In [137]:
y_pred = model.predict(arr)
print(y_pred)

240/240 [==============================] - 0s 521us/step
[[  1.5515206   10.008383   119.97869   ]
 [  1.0948755   10.009536   119.9788    ]
 [  0.93516886  10.009937   119.97884   ]
 ...
 [  4.519638    10.000893   119.9781    ]
 [  0.905674    10.010014   119.97884   ]
 [  1.5530804   10.008379   119.97869   ]]


In [138]:
predicciones = pd.DataFrame(y_pred)
predicciones

,0,1,2
0,1.551521,10.008383,119.978691
1,1.094875,10.009536,119.978798
2,0.935169,10.009937,119.978844
3,0.925050,10.009963,119.978828
4,1.190289,10.009294,119.978767
...,...,...,...
7675,1.096004,10.009532,119.978798
7676,0.931801,10.009946,119.978828
7677,4.519638,10.000893,119.978104
7678,0.905674,10.010014,119.978844


# Renombrando las columnas

In [139]:
predicciones.columns = ['Fracción Volumen', 'Radio (nm)', 'Espesor de la película (nm)']
predicciones

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.551521,10.008383,119.978691
1,1.094875,10.009536,119.978798
2,0.935169,10.009937,119.978844
3,0.925050,10.009963,119.978828
4,1.190289,10.009294,119.978767
...,...,...,...
7675,1.096004,10.009532,119.978798
7676,0.931801,10.009946,119.978828
7677,4.519638,10.000893,119.978104
7678,0.905674,10.010014,119.978844


# Redondeando los valores

normal

In [140]:
redondeo_arriba = predicciones.round()
redondeo_arriba

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,2.0,10.0,120.0
1,1.0,10.0,120.0
2,1.0,10.0,120.0
3,1.0,10.0,120.0
4,1.0,10.0,120.0
...,...,...,...
7675,1.0,10.0,120.0
7676,1.0,10.0,120.0
7677,5.0,10.0,120.0
7678,1.0,10.0,120.0


hacia arriba

In [141]:
redondeo_ceil =  predicciones.applymap(lambda x: math.ceil(x))
redondeo_ceil = redondeo_ceil.astype(float)
redondeo_ceil

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,2.0,11.0,120.0
1,2.0,11.0,120.0
2,1.0,11.0,120.0
3,1.0,11.0,120.0
4,2.0,11.0,120.0
...,...,...,...
7675,2.0,11.0,120.0
7676,1.0,11.0,120.0
7677,5.0,11.0,120.0
7678,1.0,11.0,120.0


hacia abajo

In [142]:
redondeo_abajo = predicciones.apply(np.floor)
redondeo_abajo

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,1.0,10.0,119.0
1,1.0,10.0,119.0
2,0.0,10.0,119.0
3,0.0,10.0,119.0
4,1.0,10.0,119.0
...,...,...,...
7675,1.0,10.0,119.0
7676,0.0,10.0,119.0
7677,4.0,10.0,119.0
7678,0.0,10.0,119.0


In [143]:
test_df

,Longitud de onda,Índice de absorción,Fracvol,Radio (nm),Espesor de la película (nm)
27452,492,0.006226,5,10,120
15009,831,0.001993,1,26,120
8124,700,0.000512,1,26,120
38103,748,0.000418,11,10,120
25872,432,0.002877,1,10,65
...,...,...,...,...,...
3746,820,0.002003,7,10,120
18195,422,0.000481,1,7,120
37286,551,0.033743,4,10,120
15361,703,0.000239,1,29,120


# Probando la accurancy del modelo

Primero lo haremos por cada parámetro. Se le restará los valores predichos por la red a los valores originales de test. Se contarán el número de ceros y ese será lo acertada que es nuestra red. De tener valores inferiores a 80% en número de ceros este rango se incrementará hasta llegar a un valor que el 80% de los valores sean menores a este

# Fracción Volumen

Redondeo abajo fracción volumen

In [144]:
redondeo_abajo_fracvol = redondeo_abajo['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_abajo_fracvol - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en fracción volumen")

presición del  48.736979166666664 %. Con el redondeo para abajo en fracción volumen


In [145]:
num_zeros

3743

Umbral de 1

In [146]:
#quiere decir que el 79.60% de los datos son (valor absoluto) 1 o 0 después de restar el test con el predicciones
umbral = 1
num = np.count_nonzero(np.abs(dif) <= umbral)
porcentaje = num*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en fracción volumen y un rango de 1")

presición del  78.19010416666667 %. Con el redondeo para abajo en fracción volumen y un rango de 1


In [108]:
nums = []
for num in dif:
    if np.abs(num) <= 1:
        nums.append(num)
len(nums)

5345

Redondeo arriba fracción volumen

In [147]:
redondeo_arriba_fracvol = redondeo_arriba['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_arriba_fracvol - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para arriba en fracción volumen")

presición del  66.796875 %. Con el redondeo para arriba en fracción volumen


In [148]:
num_zeros

5130

Se agrega umbral hasta llegar a 80% de aceptación 

In [149]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en fracción volumen y un rango de", umbral)
    i = i+1

presición del  78.046875 %. Con el redondeo para arriba en fracción volumen y un rango de 1
presición del  80.76822916666667 %. Con el redondeo para arriba en fracción volumen y un rango de 2


Redondeo con ceil

In [150]:
redondeo_ceil_fracvol = redondeo_ceil['Fracción Volumen'].to_numpy()
test_df_fracvol = test_df['Fracvol'].to_numpy()
dif = redondeo_ceil_fracvol - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo con ceil en fracción volumen")

presición del  26.966145833333332 %. Con el redondeo con ceil en fracción volumen


Se agrega umbral hasta llegar a 80% de aceptación 

In [151]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo ceil en fracción volumen y un rango de", umbral)
    i = i+1

presición del  77.5 %. Con el redondeo ceil en fracción volumen y un rango de 1
presición del  81.82291666666667 %. Con el redondeo ceil en fracción volumen y un rango de 2


# Radio

Redondeo abajo radio

In [152]:
redondeo_abajo_radio = redondeo_abajo['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_abajo_radio - test_df_radio
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en radio")

presición del  51.015625 %. Con el redondeo para abajo en radio


Se agrega umbral hasta llegar a 80% de aceptación 

In [153]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para abajo en radio y un rango de", umbral)
    i = i+1

presición del  55.234375 %. Con el redondeo para abajo en radio y un rango de 1
presición del  58.138020833333336 %. Con el redondeo para abajo en radio y un rango de 2
presición del  61.393229166666664 %. Con el redondeo para abajo en radio y un rango de 3
presición del  64.62239583333333 %. Con el redondeo para abajo en radio y un rango de 4
presición del  67.8515625 %. Con el redondeo para abajo en radio y un rango de 5
presición del  69.58333333333333 %. Con el redondeo para abajo en radio y un rango de 6
presición del  71.2109375 %. Con el redondeo para abajo en radio y un rango de 7
presición del  72.87760416666667 %. Con el redondeo para abajo en radio y un rango de 8
presición del  74.27083333333333 %. Con el redondeo para abajo en radio y un rango de 9
presición del  75.6640625 %. Con el redondeo para abajo en radio y un rango de 10
presición del  77.27864583333333 %. Con el redondeo para abajo en radio y un rango de 11
presición del  78.90625 %. Con el redondeo para abajo en 

Redondeo arriba radio

In [154]:
redondeo_arriba_radio = redondeo_arriba['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_arriba_radio - test_df_radio
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio")

presición del  52.265625 %. Con el redondeo para arriba en radio


Se agrega Umbral hasta llegar a 80% de aceptación

In [155]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio y un rango de", umbral)
    i = i+1

presición del  55.234375 %. Con el redondeo para arriba en radio y un rango de 1
presición del  58.138020833333336 %. Con el redondeo para arriba en radio y un rango de 2
presición del  61.393229166666664 %. Con el redondeo para arriba en radio y un rango de 3
presición del  64.62239583333333 %. Con el redondeo para arriba en radio y un rango de 4
presición del  67.8515625 %. Con el redondeo para arriba en radio y un rango de 5
presición del  69.58333333333333 %. Con el redondeo para arriba en radio y un rango de 6
presición del  71.2109375 %. Con el redondeo para arriba en radio y un rango de 7
presición del  72.87760416666667 %. Con el redondeo para arriba en radio y un rango de 8
presición del  74.27083333333333 %. Con el redondeo para arriba en radio y un rango de 9
presición del  75.6640625 %. Con el redondeo para arriba en radio y un rango de 10
presición del  77.27864583333333 %. Con el redondeo para arriba en radio y un rango de 11
presición del  78.90625 %. Con el redondeo par

Redondeo con ceil

In [157]:
redondeo_ceil_radio = redondeo_ceil['Radio (nm)'].to_numpy()
test_df_radio = test_df['Radio (nm)'].to_numpy()
dif = redondeo_ceil_radio - test_df_fracvol
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo el ceil en radio")

presición del  1.796875 %. Con el redondeo el ceil en radio


In [158]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en radio y un rango de", umbral)
    i = i+1

presición del  4.8046875 %. Con el redondeo para arriba en radio y un rango de 1
presición del  8.033854166666666 %. Con el redondeo para arriba en radio y un rango de 2
presición del  11.145833333333334 %. Con el redondeo para arriba en radio y un rango de 3
presición del  14.153645833333334 %. Con el redondeo para arriba en radio y un rango de 4
presición del  17.122395833333332 %. Con el redondeo para arriba en radio y un rango de 5
presición del  18.776041666666668 %. Con el redondeo para arriba en radio y un rango de 6
presición del  20.442708333333332 %. Con el redondeo para arriba en radio y un rango de 7
presición del  22.044270833333332 %. Con el redondeo para arriba en radio y un rango de 8
presición del  23.697916666666668 %. Con el redondeo para arriba en radio y un rango de 9
presición del  100.0 %. Con el redondeo para arriba en radio y un rango de 10


# Espesor

Redondeo abajo espesor

In [160]:
redondeo_abajo_espesor = redondeo_abajo['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_abajo_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor")

presición del  0.0 %. Con el redondeo para abajo en espesor


Se agrega Umbral hasta llegar a 80% de aceptación

In [161]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para abajo en espesor y un rango de 3
presición del  77.56510416666667 %. Con el redondeo para abajo en espesor y un rango de 4
presición del  77.56510416666667 %. Con el redondeo para abajo en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para abajo en espesor y un rango de 8
presición del  80.88541666666667 %. Con el redondeo para abajo en espesor y un rango de 9


Redondeo hacia arriba espesor

In [162]:
redondeo_arriba_espesor = redondeo_arriba['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_arriba_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo para abajo en espesor")

presición del  76.10677083333333 %. Con el redondeo para abajo en espesor


Se agrega Umbral hasta llegar a 80% de aceptación

In [163]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 3
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 4
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 8
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 9
presición del  82.60416666666667 %. Con el redondeo para arriba en espesor y un rango de 10


Redondeo con ceil

In [164]:
redondeo_ceil_espesor = redondeo_ceil['Espesor de la película (nm)'].to_numpy()
test_df_espesor = test_df['Espesor de la película (nm)'].to_numpy()
dif = redondeo_ceil_espesor - test_df_espesor
#count_nonzero() cuenta el número de elementos distintos de cero en el arreglo, mientras que la función size() devuelve el tamaño total del arreglo. 
total = dif.size
num_zeros = dif.size - np.count_nonzero(dif)
porcentaje = num_zeros*100/total
print("presición del ", porcentaje, "%. Con el redondeo ceil abajo en espesor")

presición del  76.10677083333333 %. Con el redondeo ceil abajo en espesor


Se agrega Umbral hasta llegar a 80% de aceptación

In [165]:
i = 1
while(porcentaje < 80):
    umbral = i
    num = np.count_nonzero(np.abs(dif) <= umbral)
    porcentaje = num*100/total
    print("presición del ", porcentaje, "%. Con el redondeo para arriba en espesor y un rango de", umbral)
    i = i+1

presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 1
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 2
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 3
presición del  76.10677083333333 %. Con el redondeo para arriba en espesor y un rango de 4
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 5
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 6
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 7
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 8
presición del  79.2578125 %. Con el redondeo para arriba en espesor y un rango de 9
presición del  82.60416666666667 %. Con el redondeo para arriba en espesor y un rango de 10


# Colectivo

Ajustando la columna test para poder tenerla igual que las de redondeo

In [126]:
test2 = test_df.drop(['Longitud de onda', 'Índice de absorción'], axis=1)
test2 = test2.astype(float)
nuevo = {'Fracvol': 'Fracción Volumen'}
test2 = test2.rename(columns = nuevo)
test2 = test2.reset_index()
test2 = test2.drop(['index'], axis = 1)
test2

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,5.0,10.0,120.0
1,1.0,26.0,120.0
2,1.0,26.0,120.0
3,11.0,10.0,120.0
4,1.0,10.0,65.0
...,...,...,...
7675,7.0,10.0,120.0
7676,1.0,7.0,120.0
7677,4.0,10.0,120.0
7678,1.0,29.0,120.0


In [127]:
dif_abajo = test2 - redondeo_abajo
dif_abajo

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,2.0,-5.0,6.0
1,-1.0,11.0,6.0
2,-1.0,11.0,6.0
3,9.0,-5.0,6.0
4,-1.0,-5.0,-49.0
...,...,...,...
7675,5.0,-5.0,6.0
7676,-1.0,-8.0,6.0
7677,-3.0,-1.0,5.0
7678,-1.0,14.0,6.0


Queremos que elimine los renglones que tengan 3 0's

In [128]:
nans3 = ~(dif_abajo == 0).all(axis=1)
df_sin_ceros = dif_abajo[nans3]
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,2.0,-5.0,6.0
1,-1.0,11.0,6.0
2,-1.0,11.0,6.0
3,9.0,-5.0,6.0
4,-1.0,-5.0,-49.0
...,...,...,...
7675,5.0,-5.0,6.0
7676,-1.0,-8.0,6.0
7677,-3.0,-1.0,5.0
7678,-1.0,14.0,6.0


In [129]:
df_sin_ceros.equals(dif_abajo)

True

Queremos que elimine los que tengan 2 0's

In [130]:
nans2 = dif_abajo
# Definimos una tolerancia
tolerancia = 1e-9
# Creamos una nueva columna que contenga el número de ceros por fila
nans2['zeros'] = (np.isclose(nans2, 0.0, atol=tolerancia)).sum(axis=1)
df_sin_ceros = nans2[nans2['zeros'] < 2]
df_sin_ceros = df_sin_ceros.drop(columns=['zeros'])
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,2.0,-5.0,6.0
1,-1.0,11.0,6.0
2,-1.0,11.0,6.0
3,9.0,-5.0,6.0
4,-1.0,-5.0,-49.0
...,...,...,...
7675,5.0,-5.0,6.0
7676,-1.0,-8.0,6.0
7677,-3.0,-1.0,5.0
7678,-1.0,14.0,6.0


In [131]:
nans2.equals(dif_abajo)

True

Queremos que elimine los que tenga un NaN

In [132]:
nan1 = dif_abajo
df_sin_ceros = nan1.loc[(nan1 != 0).all(axis=1)]
df_sin_ceros

,Fracción Volumen,Radio (nm),Espesor de la película (nm),zeros


In [133]:
df_sin_ceros.equals(dif_abajo)

False

Redondeo hacia arriba

In [ ]:
dif_arriba = redondeo_arriba - test2
dif_arriba

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,2.0,-6.0,-6.0
...,...,...,...
38357,NaN,NaN,NaN
38373,NaN,NaN,NaN
38375,NaN,NaN,NaN
38378,NaN,NaN,NaN


In [ ]:
dif_arriba = dif_abajo.dropna(how= 'all')
dif_arriba

,Fracción Volumen,Radio (nm),Espesor de la película (nm)
4,1.0,-6.0,-6.0
6,1.0,-18.0,-6.0
7,-13.0,5.0,-6.0
13,-6.0,5.0,-6.0
23,1.0,5.0,-16.0
...,...,...,...
7656,1.0,-13.0,-6.0
7665,1.0,7.0,-6.0
7672,1.0,5.0,-21.0
7678,-1.0,5.0,-6.0


# Función que evalua 2 tipos de errores

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
def evaluar_red_neuronal(y_pred, y_true):
    # Convertir los DataFrames a numpy arrays
    y_pred = y_pred.values
    y_true = y_true.values
    
    # Calcular el MSE y el R2
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Imprimir los resultados
    print("MSE: ", mse)
    print("R2: ", r2)
    
    # Devolver el MSE y el R2 como una tupla
    return (mse, r2)

In [ ]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Espesor de la película (nm)'], test2['Espesor de la película (nm)'])


MSE:  229.42565104166667
R2:  -0.0005251041711837878


In [ ]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Fracción Volumen'], test2['Fracción Volumen'])


MSE:  15.985416666666667
R2:  -0.028586265256485666


In [ ]:
mse, r2 = evaluar_red_neuronal(redondeo_abajo['Radio (nm)'], test2['Radio (nm)'])


MSE:  69.266015625
R2:  0.000445180446051463
